# Preprocessing Steps for Head Pose Videos

This notebook is for extracting mesh features from `.webm` videos generated by the head pose recording experiment. The result of running this pipeline is a `.json` file for each subject/session in the head pose experiment.

In [ ]:
import numpy as np
import cv2
import os
import fnmatch
import json
!pip install mediapipe
import mediapipe as mp
from google.colab import drive
drive.mount('/content/drive')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Configure MediaPipe FaceMesh


In [ ]:
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=True, refine_landmarks=True)

# Process Video Pipeline

Define the function to take in a `path` to a video file and output an array of shape (`frames`, `landmarks`, `3`), where `frames` is the number of frames in the video file, `landmarks` is the number of landmarks extracted by the face mesh model, and `3` corresponds to the `x`, `y`, and `z` coordinates for each landmark.

In [ ]:
def extract_mesh_from_video(path):
  # open a video file for video capturing
  cap = cv2.VideoCapture(path)
  out = []
  
  # to see if video capturing has been initialized
  while(cap.isOpened()):
    # return (1) if any frames grabbed (2) grabbed image (empty if ret is false)
    ret, frame = cap.read()
    # Q: why frame could be none?
    if frame is not None:
      # process an RGB image and returns the face landmarks on each detected face
      results = face_mesh.process(frame)
      # check if any faces detected
      if not results.multi_face_landmarks:
        continue
      landmarks = results.multi_face_landmarks[0].landmark
      # store landmarks as an array of arrays
      lm = [[a.x, a.y, a.z] for a in landmarks]
      # 3D tensor that stores landmarks frame by frame
      out.append(lm)
    else:
      break

  if len(out) > 0:
    out = np.reshape(np.array(out), (len(out), -1, 3)).tolist()
  return out

Run through all of the video files in the folder and process their data. If `.json` output already exists for the subject ID, then skip processing. (To overwrite existing data, delete the corresponding `.json` file before running this step.)

In [ ]:
# get the path of the webm file
path = '/content/drive/Shareddrives/URSI 2022/Eye Tracking ML/prolific_eye_data_experiment/vae_test_webm/'

# store all the file directories
all_files = os.listdir(path)
print(len(all_files))

# get unique subjects
unique_subjects = set([filepath.split('_')[0] for filepath in os.listdir(path)])
print(unique_subjects)

for subject in unique_subjects:
  all_data = {}

  print(subject)

  # check if the subject json file already exists. if so, skip the remainning body
  if os.path.isfile('/content/drive/Shareddrives/URSI 2022/Eye Tracking ML/prolific_eye_data_experiment/vae_test_json/'+subject+'.json'):
    continue
   
  subject_data = []
  # go through all file directories in the webm file, find those that start with the subject name
  subject_files = fnmatch.filter(all_files, subject+'*')
  # manage every single file directory that starts with the subject name
  for filename in subject_files:
    # transform file name into an array
    fileinfo = filename.replace('.','_').split('_')
    # store relevant values
    subject = fileinfo[0]
    meshfeatures = extract_mesh_from_video(path + filename)
    # create and append a dictionary to the exisiting array
    subject_data.append({
        'features': meshfeatures 
    })
  # once the last for loop is over, assign the subject_data array as the value to the subject key
  all_data[subject] = subject_data

  # export the json file for the subject to the drive
  with open('/content/drive/Shareddrives/URSI 2022/Eye Tracking ML/prolific_eye_data_experiment/vae_test_json/'+subject+'.json', 'w') as file:
    json.dump(all_data, file)

17
{'8wplohk7', 'duavtnj5', 'q2rv1bj0', '0647xw1u', 'ryvtw1w8', 'dl75d10s', 'c17pt575', '1nqsmw2a', 'z98nk7pa', 'epsxz1ys', '76qjhxwz', '459wtwlr', '4n48phr9', 'to5p13vd', '7ladzmrj', '8vf1z9hb'}
8wplohk7
duavtnj5
q2rv1bj0
0647xw1u
ryvtw1w8
dl75d10s
c17pt575
1nqsmw2a
z98nk7pa
epsxz1ys
76qjhxwz
459wtwlr
4n48phr9
to5p13vd
7ladzmrj
8vf1z9hb


In [ ]:
unique_subjects

{'0647xw1u',
 '1nqsmw2a',
 '459wtwlr',
 '4n48phr9',
 '76qjhxwz',
 '7ladzmrj',
 '8vf1z9hb',
 '8wplohk7',
 'c17pt575',
 'dl75d10s',
 'duavtnj5',
 'epsxz1ys',
 'q2rv1bj0',
 'ryvtw1w8',
 'to5p13vd',
 'z98nk7pa'}